In [1]:
import os
os.chdir('D:/mesh-and-bones-to-rig') # Change to the right directory.

In [2]:
import trimesh
import numpy as np

import ipywidgets as widgets

import torch
import torch.nn as nn
import torch.nn.init as init
from torch_geometric.loader import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [3]:
from data.dataset import parse_rig_info
from data.dataset import MeshBonesToRigDataset
from noteboorks.training_utils import train_epoch, train_model
from noteboorks.visualization_utils import display_view

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
# Assume the dataset preprocessed files are stored in this folder.
root_dir = "./data/model_data/ModelResource_MeshAndBonesToRig_preproccessed_reduced_subset/"
cache_dir = os.path.join(root_dir, "precomputed")

Now can we overfit a 1 mesh? We will choose the model with name 9811.

In [5]:
mesh_name = "9811"

In [6]:
# It is just a single mesh so I can go without the dataloader.
train_dataset_one_mesh = MeshBonesToRigDataset(root_dir=root_dir, cache_dir=cache_dir, k=8, allowed_names=[mesh_name])

In [7]:
data_mesh = train_dataset_one_mesh.__getitem__(0)
skinning_weights = data_mesh["target_skin_weights"]
skinning_weights_np = skinning_weights.cpu().numpy()

In [8]:
skinning_weights.shape

torch.Size([2811, 24])

In [9]:
skinning_weights

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.7500],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

Target skin weights sample stats.

In [10]:
skinning_weights.mean(), skinning_weights.std()

(tensor(0.0417), tensor(0.1866))

In [11]:
overfit_mesh = trimesh.load(os.path.join(root_dir, "obj_remesh", f"{mesh_name}.obj"))

In [12]:
overfit_mesh.show() # pokemon like mesh

In [13]:
# We will need it there for later inference.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_mesh.to(device)

Data(vertices=[2811, 3], edge_index_geodesic=[2, 22488], edge_attr_geodesic=[22488, 1], vertex_neighbors=[2811, 8], vertex_adj=[2811, 2811], vertex_normals=[2811, 3], bone_positions=[24, 3], bone_features=[24, 8], bone_adj=[24, 24], target_skin_weights=[2811, 24], volumetric_geodesic=[2811, 24], surface_geodesic=[2811, 2811], num_nodes=2811)

In [14]:
# All hyperparameters at once
hyperparams = {
    # Model Architecture
    'mesh_encoder_in_channels': 3,
    'mesh_encoder_hidden_channels': 128,
    'mesh_encoder_out_channels': 256,
    'mesh_encoder_kernel_size': 5,
    'mesh_encoder_num_layers': 3,
    'mesh_encoder_dim': 1,
    'bone_encoder_in_channels': 8,
    'bone_encoder_hidden_channels': 64,
    'bone_encoder_out_channels': 64,
    'bone_encoder_num_layers': 2,
    'fusion_common_dim': 128,
    'fusion_top_k': 4, # Ususally at most 4 bones influence a vertex
    'fusion_alpha': 1.0, # Learnable, we are giving it a default value
    'fusion_alpha_learnable': True,
    'refinement_gamma': 0.5,
    'with_refinement': False,

    # Training Settings
    'mesh_name': "9811",
    'num_epochs': 500, # I want to see how it performs after 500 epochs.
    'batch_size': 1, # Set batch size to 1 as the problem with staking multiple Data objects with tensors of variable dimensions is still not resolved.
    'learning_rate': 1e-4,  # Set learning rate as needed (in the Rignet paper they used this learning rate).
    'optimizer': 'Adam',
    'initialization': 'xavier_uniform_', # I wanted to try out a different initialization scheme.

    # Dataset Settings
    'k_neighbors': 8, # Up to k nearest neighbours

    # Loss Settings
    'lambda_skin': 1.0,
    'lambda_geo': 1.0,
    'lambda_geo_alpha': 1.0,
    'lambda_smooth': 1.0
}

In [15]:
model = None
model = train_model(model, train_dataset_one_mesh, hyperparams, "runs/one_mesh_overfit_training")

Parameter containing:
tensor(1., device='cuda:0', requires_grad=True)
Loss Skin: 2.109611, Loss Geo: 12.403839, Loss Smooth: 0.000408, Total loss: 14.513858
Epoch 1/500 - Train Loss: 14.5139
Parameter containing:
tensor(1.0001, device='cuda:0', requires_grad=True)
Loss Skin: 2.030324, Loss Geo: 11.948736, Loss Smooth: 0.000400, Total loss: 13.979461
Epoch 2/500 - Train Loss: 13.9795
Parameter containing:
tensor(1.0002, device='cuda:0', requires_grad=True)
Loss Skin: 1.987326, Loss Geo: 11.460524, Loss Smooth: 0.000396, Total loss: 13.448246
Epoch 3/500 - Train Loss: 13.4482
Parameter containing:
tensor(1.0003, device='cuda:0', requires_grad=True)
Loss Skin: 1.961102, Loss Geo: 10.942168, Loss Smooth: 0.000398, Total loss: 12.903668
Epoch 4/500 - Train Loss: 12.9037
Parameter containing:
tensor(1.0004, device='cuda:0', requires_grad=True)
Loss Skin: 1.949036, Loss Geo: 10.398738, Loss Smooth: 0.000403, Total loss: 12.348178
Epoch 5/500 - Train Loss: 12.3482
Parameter containing:
tensor(

In [16]:
%load_ext tensorboard

In [17]:
%tensorboard --logdir ./runs

Reusing TensorBoard on port 6007 (pid 54576), started 0:37:08 ago. (Use '!kill 54576' to kill it.)

Let's see how the model performs on the mesh we tried to overfit it with.

In [18]:
model.eval()
predicted_skinning_weights = model(data_mesh["vertices"],
                                    data_mesh["edge_index_geodesic"],
                                    data_mesh["edge_attr_geodesic"],
                                    data_mesh["vertex_neighbors"],
                                    data_mesh["vertex_adj"],
                                    data_mesh["vertex_normals"],
                                    data_mesh["bone_features"],
                                    data_mesh["bone_adj"],
                                    data_mesh["volumetric_geodesic"],
                                    data_mesh["surface_geodesic"])
predicted_skinning_weights_np = predicted_skinning_weights.cpu().detach().numpy()

Parameter containing:
tensor(1.0104, device='cuda:0', requires_grad=True)


I would like to visualize the ground truth predicted skinning weights for each bone side by side.

In [19]:
bone_positions_np, root_joint, bone_hierarchy, skin_weights_dict, bone_names = parse_rig_info(os.path.join(root_dir, "rig_info_remesh", f"{mesh_name}.txt")) # I need only the bone names in reallity for the dropdown.
bone_options = [(name, i) for i, name in enumerate(bone_names)]

In [20]:
dropdown = widgets.Dropdown(
    options=bone_options,
    value=0,  # default selected integer value
    description='Bone Name:'
)

In [21]:
display_view(overfit_mesh, skinning_weights_np, predicted_skinning_weights_np, dropdown, side_view=widgets.fixed(False))

In [22]:
display_view(overfit_mesh, skinning_weights_np, predicted_skinning_weights_np, dropdown, side_view=widgets.fixed(True))

As you can see the left one is the target skinning and the right one is the predicted one.

The combined loss function is strange. The Geodesic loss increases at times while the Skinning loss and Smoothness loss decrease. Maybe the reason the Geodesic loss increases is that the $\alpha$ I use in the model is a learnavble parameter while the alpha used in the loss function is a constant (for now set to 1.0). Surely if I set the model's $\alpha$ to 1.0 the Geodesic loss will decrease also?

In [23]:
hyperparams['fusion_alpha_learnable'] = False

In [24]:
model_1 = None
model_1 = train_model(model_1, train_dataset_one_mesh, hyperparams, "runs/one_mesh_overfit_training")
model_1.eval()
predicted_skinning_weights_1 = model_1(data_mesh["vertices"],
                                    data_mesh["edge_index_geodesic"],
                                    data_mesh["edge_attr_geodesic"],
                                    data_mesh["vertex_neighbors"],
                                    data_mesh["vertex_adj"],
                                    data_mesh["vertex_normals"],
                                    data_mesh["bone_features"],
                                    data_mesh["bone_adj"],
                                    data_mesh["volumetric_geodesic"],
                                    data_mesh["surface_geodesic"])
predicted_skinning_weights_np_1 = predicted_skinning_weights_1.cpu().detach().numpy()

Parameter containing:
tensor(1., device='cuda:0')
Loss Skin: 1.969418, Loss Geo: 12.403789, Loss Smooth: 0.000388, Total loss: 14.373595
Epoch 1/500 - Train Loss: 14.3736
Parameter containing:
tensor(1., device='cuda:0')
Loss Skin: 1.941004, Loss Geo: 11.948567, Loss Smooth: 0.000390, Total loss: 13.889961
Epoch 2/500 - Train Loss: 13.8900
Parameter containing:
tensor(1., device='cuda:0')
Loss Skin: 1.920659, Loss Geo: 11.461122, Loss Smooth: 0.000389, Total loss: 13.382170
Epoch 3/500 - Train Loss: 13.3822
Parameter containing:
tensor(1., device='cuda:0')
Loss Skin: 1.912097, Loss Geo: 10.944674, Loss Smooth: 0.000374, Total loss: 12.857144
Epoch 4/500 - Train Loss: 12.8571
Parameter containing:
tensor(1., device='cuda:0')
Loss Skin: 1.899871, Loss Geo: 10.403211, Loss Smooth: 0.000378, Total loss: 12.303459
Epoch 5/500 - Train Loss: 12.3035
Parameter containing:
tensor(1., device='cuda:0')
Loss Skin: 1.887867, Loss Geo: 9.844265, Loss Smooth: 0.000375, Total loss: 11.732507
Epoch 6/5

In [25]:
dropdown_1 = widgets.Dropdown(
    options=bone_options,
    value=0,  # default selected integer value
    description='Bone Name:'
)

In [26]:
display_view(overfit_mesh, skinning_weights_np, predicted_skinning_weights_np_1, dropdown_1, side_view=widgets.fixed(False))

In [27]:
display_view(overfit_mesh, skinning_weights_np, predicted_skinning_weights_np_1, dropdown_1, side_view=widgets.fixed(True))

I guess that at times it just jitters. I bet that if the learnable $\alpha$ took more inetersing values than 1.0  it would be the reason why at times the Geodesic loss increases more than it should.

Now here is the reason that for now I do not use the refinement module:

In [28]:
hyperparams['with_refinement'] = True

In [29]:
model_2 = None
model_2 = train_model(model_2, train_dataset_one_mesh, hyperparams, "runs/one_mesh_overfit_training")
model_2.eval()
predicted_skinning_weights_2 = model_2(data_mesh["vertices"],
                                    data_mesh["edge_index_geodesic"],
                                    data_mesh["edge_attr_geodesic"],
                                    data_mesh["vertex_neighbors"],
                                    data_mesh["vertex_adj"],
                                    data_mesh["vertex_normals"],
                                    data_mesh["bone_features"],
                                    data_mesh["bone_adj"],
                                    data_mesh["volumetric_geodesic"],
                                    data_mesh["surface_geodesic"])
predicted_skinning_weights_np_2 = predicted_skinning_weights_2.cpu().detach().numpy()

Parameter containing:
tensor(1., device='cuda:0')
Loss Skin: 2.818593, Loss Geo: 0.004349, Loss Smooth: 0.000000, Total loss: 2.822942
Epoch 1/500 - Train Loss: 2.8229
Parameter containing:
tensor(1., device='cuda:0')
Loss Skin: 2.814645, Loss Geo: 0.003592, Loss Smooth: 0.000000, Total loss: 2.818237
Epoch 2/500 - Train Loss: 2.8182
Parameter containing:
tensor(1., device='cuda:0')
Loss Skin: 2.810748, Loss Geo: 0.013806, Loss Smooth: 0.000000, Total loss: 2.824554
Epoch 3/500 - Train Loss: 2.8246
Parameter containing:
tensor(1., device='cuda:0')
Loss Skin: 2.807218, Loss Geo: 0.034949, Loss Smooth: 0.000000, Total loss: 2.842167
Epoch 4/500 - Train Loss: 2.8422
Parameter containing:
tensor(1., device='cuda:0')
Loss Skin: 2.805073, Loss Geo: 0.066886, Loss Smooth: 0.000000, Total loss: 2.871959
Epoch 5/500 - Train Loss: 2.8720
Parameter containing:
tensor(1., device='cuda:0')
Loss Skin: 2.803421, Loss Geo: 0.109263, Loss Smooth: 0.000000, Total loss: 2.912683
Epoch 6/500 - Train Loss:

In [30]:
dropdown_2 = widgets.Dropdown(
    options=bone_options,
    value=2,  # default selected integer value: L_Feeler_01
    description='Bone Name:'
)

In [31]:
display_view(overfit_mesh, skinning_weights_np, predicted_skinning_weights_np_2, dropdown_2, side_view=widgets.fixed(False))

In [32]:

display_view(overfit_mesh, skinning_weights_np, predicted_skinning_weights_np_2, dropdown_2, side_view=widgets.fixed(True))

The loss functions doesn't want to converge to zero. If you click on L_Feeler_01 you can see that the refinement module is not working well. It messes up the skinning weights for some bones.

As a whole this model would need a lot of tunning of the hyper parameters to work well. Alas, we will see when I will have the time to play with optuna for that.